### Introduction

Plavcik's journey simulation from Plavcik and Vratko fairytale. Description of the code with comments here: link

### Process

Variables:
1. Starting date time.
2. Starting location.
3. How many days in the journey.
4. How often will Plavcik check for sun's azimuth.
5. How fast can person walk.


### Geolocating places

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="plavcik_vratko")
location = geolocator.geocode("Vancouver,BC")
print(location.longitude, location.latitude)

### Calculation of the new position based on azimuth and distance

In [ ]:
from pygc import great_circle

great_circle(distance=111000, azimuth=65, latitude=30, longitude=-74)

### Getting azimuth and altitude of the sun, get important times

In [ ]:
from suncalc import get_position, get_times
from datetime import datetime

date = datetime.now()
lon = -123.113952
lat = 49.2608724

position_data = get_position(date, lon, lat)
times_data = get_times(date, lon, lat)

position_data

In [ ]:
get_times(date, lon, lat)

### Setup

In [1]:
from datetime import datetime
import numpy
import pandas as pd
import geopandas
from geopy import distance

from pygc import great_circle

from suncalc import get_position, get_times
import math

import folium
from folium.features import DivIcon

In [2]:
def get_sun_azimuth(actual_datetime, actual_lon, actual_lat):
    data = get_position(actual_datetime, actual_lon, actual_lat)
    azimuth = (data['azimuth'] * 180 / math.pi) + 180
    
    return azimuth 

In [61]:
def get_sun_time(input_date, actual_lon, actual_lat):
    times = get_times(input_date, actual_lon, actual_lat)

    return times['sunrise'], times['sunset']

In [4]:
def create_map(gdf, name):

    coords_1 = (gdf['geometry'].iloc[0].y, gdf['geometry'].iloc[0].x)
    coords_2 = (gdf['geometry'].iloc[-1].y, gdf['geometry'].iloc[-1].x)

    total_distance = round(distance.distance(coords_1, coords_2).km, 2)
    distance_traveled = gdf['traveled'].sum()/1000

    map_latitude = gdf['geometry'].y.mean()
    map_longitude = gdf['geometry'].x.mean()

    m = folium.Map(location=[map_latitude, map_longitude])

    for _, row in gdf.iterrows():
        folium.Marker(location=[row['geometry'].y, row['geometry'].x],
                      popup=row['new_time'], tooltip=row['new_time']).add_to(m)

    folium.Circle([start_lat, start_lon], radius=total_distance*1000).add_to(m)

    text1 = "time: {}".format(key)
    text2 = "Total {} km @ {} km travelled.".format(
        total_distance, distance_traveled)
    folium.map.Marker([start_lat, start_lon], icon=DivIcon(icon_size=(300, 36), icon_anchor=(300, 36),
                                                           html='<div style="font-size: 20pt">{}\n{}</div>'.format(
                                                               text1, text2),
                                                           )).add_to(m)

    sw = gdf[['new_lat', 'new_lon']].min().values.tolist()
    ne = gdf[['new_lat', 'new_lon']].max().values.tolist()
    m.fit_bounds([sw, ne])

    m.save("{}.html".format(name))
    return display(m)

### Times of interest

In [5]:
dates_dict = dict()
dates_dict['spring_equinox'] = '2022-03-20'
dates_dict['summer_solstice'] = '2022-06-21'
dates_dict['autumn_equinox'] = '2022-09-23'
dates_dict['winter_solstice'] = '2022-12-21'

dates_dict

{'spring_equinox': '2022-03-20',
 'summer_solstice': '2022-06-21',
 'autumn_equinox': '2022-09-23',
 'winter_solstice': '2022-12-21'}

### Place of interest

In [104]:
pl_of_interest = dict()
pl_of_interest['center_of_europe'] = (48.743774, 18.913666)
pl_of_interest['arctic_circle_yukon'] = (66.566667, -141)
pl_of_interest['tropic_capricorn_australia'] = (-23.441889, 133.832804)
pl_of_interest['equator_uganda'] = (0, 29.716667)


m = folium.Map()

for key, value in pl_of_interest.items():
    folium.Marker(location=[value[0], value[1]],
                  popup=key, tooltip=key).add_to(m)
    folium.Marker(location=[value[0], value[1]],
                  icon=DivIcon(icon_size=(1, 1), icon_anchor=(1, 0),
                               html='<div style="font-size: 15pt">{}</div>'.format(key))).add_to(m)

 
display(m)

### The Code

In [106]:
journey_days = 7
corect_azimuth_min = 30
walk_kmh = 5
increment_distance_km = (walk_kmh / (60 / corect_azimuth_min) * 1000)
days_to_substract = round(journey_days/2)

location = 'equator_uganda'

start_lon = pl_of_interest[location][1]
start_lat = pl_of_interest[location][0]

for key, value in dates_dict.items():

    start_date = datetime.fromisoformat(value)
    start_date = start_date - pd.Timedelta(days=days_to_substract)

    start_sunrise = get_sun_time(start_date, start_lon, start_lat)[0]
    print("Journey started: " + str(start_date),
          "Start day sunrise: " + str(start_sunrise))

    if pd.isnull(start_sunrise):
        print("No sunrise for given time of the year for given location!")
        start_sunrise = start_date

    start_sun_azimuth = get_sun_azimuth(start_sunrise, start_lon, start_lat)
#     print("Start date sunrise azimuth: " + str(start_sun_azimuth))

    sun_azimuth = start_sun_azimuth
    new_lon = start_lon
    new_lat = start_lat
    date = start_date

    df = pd.DataFrame()

    for i in range(0, journey_days):
        journey_date = date + pd.Timedelta(days=i)

        day_sunrise = get_sun_time(journey_date, new_lon, new_lat)[0]
        if pd.isnull(day_sunrise):
            day_sunrise = journey_date

        journey_day = str(i+1)
        print("Day of the journey: " + str(journey_day), "Current date: " + str(journey_date), "Day sunrise time: " + str(day_sunrise))
        new_time = day_sunrise

        while True:
            new_pos = great_circle(distance=increment_distance_km,
                                   azimuth=sun_azimuth, latitude=new_lat, longitude=new_lon)

            new_lat = new_pos['latitude']
            new_lon = new_pos['longitude']

            new_time = new_time + pd.Timedelta(minutes=corect_azimuth_min)
            sun_azimuth = get_sun_azimuth(new_time, new_lon, new_lat)

    #         print("Updated time: " + str(new_time))
    #         print(new_lon, new_lat)
    #         print("Actual sun azimuth: " + str(sun_azimuth))

            new_loc_sunset = get_sun_time(new_time, new_lon, new_lat)[1]

            row_dict = dict(
                journey_start_date=start_date.date(),
                journey_date=journey_date.date(),
                journey_day=journey_day,
                day_sunrise=day_sunrise,
                new_time=new_time,
                new_lat=new_lat,
                new_lon=new_lon,
                sun_azimuth=sun_azimuth,
                traveled=increment_distance_km)

            df = df.append(row_dict, ignore_index=True)

            if (new_time > new_loc_sunset) or (new_time > (journey_date + pd.Timedelta(days=i))):
                break

    gdf = geopandas.GeoDataFrame(
        df, geometry=geopandas.points_from_xy(df.new_lon, df.new_lat))
    
    create_map(gdf, key)

Journey started: 2022-03-16 00:00:00 Start day sunrise: 2022-03-16 04:07:52.673866240
Day of the journey: 1 Current date: 2022-03-16 00:00:00 Day sunrise time: 2022-03-16 04:07:52.673866240
Day of the journey: 2 Current date: 2022-03-17 00:00:00 Day sunrise time: 2022-03-17 04:07:29.357074432
Day of the journey: 3 Current date: 2022-03-18 00:00:00 Day sunrise time: 2022-03-18 04:07:09.651327488
Day of the journey: 4 Current date: 2022-03-19 00:00:00 Day sunrise time: 2022-03-19 04:06:59.958561792
Day of the journey: 5 Current date: 2022-03-20 00:00:00 Day sunrise time: 2022-03-20 04:06:49.055703808
Day of the journey: 6 Current date: 2022-03-21 00:00:00 Day sunrise time: 2022-03-21 04:06:36.194498304
Day of the journey: 7 Current date: 2022-03-22 00:00:00 Day sunrise time: 2022-03-22 04:06:23.926530304


Journey started: 2022-06-17 00:00:00 Start day sunrise: 2022-06-17 03:59:40.605965824
Day of the journey: 1 Current date: 2022-06-17 00:00:00 Day sunrise time: 2022-06-17 03:59:40.605965824
Day of the journey: 2 Current date: 2022-06-18 00:00:00 Day sunrise time: 2022-06-18 03:59:46.858315776
Day of the journey: 3 Current date: 2022-06-19 00:00:00 Day sunrise time: 2022-06-19 03:59:23.823558400
Day of the journey: 4 Current date: 2022-06-20 00:00:00 Day sunrise time: 2022-06-20 03:59:10.516811008
Day of the journey: 5 Current date: 2022-06-21 00:00:00 Day sunrise time: 2022-06-21 03:58:57.129718016
Day of the journey: 6 Current date: 2022-06-22 00:00:00 Day sunrise time: 2022-06-22 03:58:43.683683584
Day of the journey: 7 Current date: 2022-06-23 00:00:00 Day sunrise time: 2022-06-23 03:58:30.200634624


Journey started: 2022-09-19 00:00:00 Start day sunrise: 2022-09-19 03:53:04.982724608
Day of the journey: 1 Current date: 2022-09-19 00:00:00 Day sunrise time: 2022-09-19 03:53:04.982724608
Day of the journey: 2 Current date: 2022-09-20 00:00:00 Day sunrise time: 2022-09-20 03:52:37.470300672
Day of the journey: 3 Current date: 2022-09-21 00:00:00 Day sunrise time: 2022-09-21 03:52:13.704990208
Day of the journey: 4 Current date: 2022-09-22 00:00:00 Day sunrise time: 2022-09-22 03:51:59.931297024
Day of the journey: 5 Current date: 2022-09-23 00:00:00 Day sunrise time: 2022-09-23 03:51:44.684990720
Day of the journey: 6 Current date: 2022-09-24 00:00:00 Day sunrise time: 2022-09-24 03:51:28.545146368
Day of the journey: 7 Current date: 2022-09-25 00:00:00 Day sunrise time: 2022-09-25 03:51:14.203844608


Journey started: 2022-12-17 00:00:00 Start day sunrise: 2022-12-17 03:54:49.251601920
Day of the journey: 1 Current date: 2022-12-17 00:00:00 Day sunrise time: 2022-12-17 03:54:49.251601920
Day of the journey: 2 Current date: 2022-12-18 00:00:00 Day sunrise time: 2022-12-18 03:55:11.778979072
Day of the journey: 3 Current date: 2022-12-19 00:00:00 Day sunrise time: 2022-12-19 03:55:05.182836992
Day of the journey: 4 Current date: 2022-12-20 00:00:00 Day sunrise time: 2022-12-20 03:55:08.397665536
Day of the journey: 5 Current date: 2022-12-21 00:00:00 Day sunrise time: 2022-12-21 03:55:11.607424768
Day of the journey: 6 Current date: 2022-12-22 00:00:00 Day sunrise time: 2022-12-22 03:55:14.829536
Day of the journey: 7 Current date: 2022-12-23 00:00:00 Day sunrise time: 2022-12-23 03:55:18.081741056
